In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

## Components
1. Initialize a component
2. Inspect expected input/output types

In [1]:
from haystack.components.embedders import OpenAIDocumentEmbedder

embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")

ValueError: None of the following authentication environment variables are set: ('OPENAI_API_KEY',)

## Pipelines (Indexing + RAG Pipelines)

1. Add components
2. Connect components

## Condiaional Branches